In [1]:
import pandas as pd
from datasets import load_dataset
import os
if not os.path.exists("../data/train.csv"):
    ds = load_dataset("traintogpb/aihub-koja-translation-integrated-large-4.3m")
    train = ds['train'].to_pandas()
    valid = ds['validation'].to_pandas()
    test = ds['test'].to_pandas()
    train.to_csv("../data/train.csv", index=False)
    valid.to_csv("../data/valid.csv", index=False)
    test.to_csv("../data/test.csv", index=False)
else:
    print("data already exists")
    train = pd.read_csv("../data/train.csv")
    valid = pd.read_csv("../data/valid.csv")
    test = pd.read_csv("../data/test.csv")

data already exists


In [ ]:
from konlpy.tag import Okt

In [ ]:
from konlpy.tag import Okt
import openai
import requests
from fugashi import Tagger
import sacrebleu
from dotenv import load_dotenv
import os

load_dotenv()

def ja_tokenizing(sentence="私は学校に行きます。"):
    tagger = Tagger()  # 기본 설정, unidic-lite 사전 사용
    tokens = [word.surface for word in tagger(sentence)]
    return tokens


def ko_tokenizing(results):
    okt = Okt()
    tokenized = []
    for result in results:
        tokens_ko = okt.morphs(result['ko'])
        tokens_tr = okt.morphs(result['translation'])
        tokenized.append((tokens_ko, tokens_tr))
    return tokenized

deepl_api_key = os.getenv("DEEPL_API_KEY")

# deepL API

def translate_with_deepL(message):
    url_for_deepL = "https://api-free.deepl.com/v2/translate"
    params = {'auth_key': deepl_api_key,  # 여기에 본인의 DeepL API 키를 입력하세요
              'text': message,
              'source_lang': 'JA',
              'target_lang': 'KO'}
    response = requests.post(url_for_deepL, data=params)
    if response.status_code == 200:
        translation = response.json()['translations'][0]['text']
        return translation
    else:
        print(f"DeepL API 요청 실패: {response.status_code}, {response.text}")
        return None

openai_api_key = os.getenv("OPENAI_API_KEY")

# GPT API
def translate_with_gpt(message):
    client = openai.OpenAI(api_key=openai_api_key)
    response = client.chat.completions.create(
        model="gpt-4.1-nano-2025-04-14",  # gpt-4o 또는 최신 지원 모델명 사용
        messages=[
            {"role": "system", "content": "앞으로 당신은 한국어를 일본어로 번역하는 역할을 합니다. 번역은 자연스럽고 정확해야 합니다."},
            {"role": "user", "content": message}
        ]
    )
    return response.choices[0].message.content

def get_bleu(tokenized):
    hypothesis = []
    references = []
    for tokens_ko, tokens_tr in tokenized:
        hypothesis.append(" ".join(tokens_tr))
        references.append(" ".join(tokens_ko))
    
    bleu = sacrebleu.corpus_bleu(hypothesis, references)
    print(f"전체 샘플 BLEU: {bleu.score:.2f}")
    return bleu


In [18]:
import numpy as np
import pandas as pd

np.random.seed(42)
test_sample = test.sample(frac=1, random_state=42).reset_index(drop=True).head(10)
models = [translate_with_gpt, translate_with_deepL]
results = []
for i in range(1):
    print(f"test_sample['ko'][{i}]: {test_sample['ko'][i]}")
    print(f"test_sample['ja'][{i}]: {test_sample['ja'][i]}")
    translation = models[0](test_sample['ja'][i])
    print(f"GPT 번역 결과: {translation}\n")
    results.append({
        "ko": test_sample['ko'][i],
        "ja": test_sample['ja'][i],
        "translation": translation
    })
    
tokenized = ko_tokenizing(results)

bleu_score = get_bleu(tokenized)

test_sample['ko'][0]: 결과는 조직 내 구성원 간의 경쟁은 항상 긍정적인 결과를 가져오는 것이 아니라 특정한 조건 하에서 긍정적으로 작용한다는 것을 시사한다.
test_sample['ja'][0]: 結果は組織内構成員間の競争は常に肯定的な結果をもたらすのではなく、特定の条件下で肯定的に作用することを示唆する。
GPT 번역 결과: 結果は、組織内の構成員間の競争が常に肯定的な結果をもたらすわけではなく、特定の条件下で肯定的に作用することを示唆している。

전체 샘플 BLEU: 0.00
